# Debug

## Bug: 
```
System 2 parameters: {'sigma_u': 9.0, 'sigma_b': 0.060145439998495594, 'rho': 1179.133813806039, 'd': 0.7827636158617441, 'label': 1}
Steady state series saved to data_12_04_2025/mRNA_trajectories_variance_3083_1200/steady_state_trajectories/m_traj_3083.9999999999986_1200.0_9_SS.csv
...
...
Attempt 1/10
Attempt 2/10
Attempt 3/10
Attempt 4/10
Attempt 5/10
Attempt 6/10
Attempt 7/10
Attempt 8/10
Attempt 9/10
Attempt 10/10
No suitable solution found after multiple attempts. Try increasing num_guesses or widening the ranges.
[STRESS] ❌ No suitable solution found.
Attempt 1/10
[NORMAL] ✅ Found: {'rho': 1179.1338138059837, 'sigma_b': 0.060145439998495545, 'd': 0.7827636158617439}
Traceback (most recent call last):
  File "/home/ianyang/stochastic_simulations/experiments/SSA_telegraph_model/var_v_accuracy_plot/var_v_accuracy_12_04_2025.py", line 92, in <module>
    "sigma_b": results["stress"]['sigma_b'], 
               ~~~~~~~^^^^^^^^^^
KeyError: 'stress'
```

## Issue: simulation.mean_var_autocorr.find_parameters

In [ ]:
# find the variance ratio that went wrong
import numpy as np
from simulation.mean_var_autocorr import find_parameters
import tqdm 
###############################################################################
# 1) Define target mean, variance and autocorrelations, and some parameters to start with
###############################################################################
variance_target_normal = 1200.0  # Fixed normal variance
mu_target = 10.0                 # Mean (same for both)
variance_ratios = np.arange(0.1, 3.0, 0.01)

# this is the ratio that went wrong
current_ratio = variance_ratios[248]
print(current_ratio)

autocorr_target = 0.5

parameters = {
    "stress": {"sigma_u": 18.0},
    "normal": {"sigma_u": 9.0}
}



###############################################################################
# 2) Loop over different variance ratios
###############################################################################
for ratio in tqdm.tqdm(variance_ratios[248:], desc="Running Variance Ratio Simulations"):
    # For the stress condition, we define variance_target_stress by ratio
    variance_target_stress = ratio * variance_target_normal

    # store results for each condition
    results = {}
    for condition, param in parameters.items():
        # Decide which variance to use for this condition
        if condition == "normal":
            var_for_condition = variance_target_normal
        else:  # condition == "stress"
            var_for_condition = variance_target_stress

        try:
            # Fix all three statistical properties
            rho, sigma_b, d = find_parameters(
                param, mu_target, var_for_condition, autocorr_target, 
                rho_range = (1,1000) , num_guesses=1000 # debug
            )

            results[condition] = {"rho": rho, "sigma_b": sigma_b, "d": d}
            print(f"[{condition.upper()}] ✅ Found: {results[condition]}")
        except ValueError as e:
            print(f'{e}')
            print(f"[{condition.upper()}] ❌ No suitable solution found.")


    # update parameter sets 
    parameter_sets = [
        {"sigma_u": parameters["stress"]["sigma_u"], 
        "sigma_b": results["stress"]['sigma_b'], 
        "rho": results["stress"]['rho'], 
        "d": results["stress"]['d'], 
        "label": 0},
        
        {"sigma_u": parameters["normal"]["sigma_u"], 
        "sigma_b": results["normal"]['sigma_b'], 
        "rho": results["normal"]['rho'], 
        "d": results["normal"]['d'], 
        "label": 1}
    ]

    # Output the results
    print("Updated Parameter Sets:", parameter_sets)

    # Simulation parameters
    min_d = min(pset['d'] for pset in parameter_sets)
    steady_state_time = int(10 / min_d)
    time_points = np.arange(0, 144.0, 1.0)
    extended_time_points = np.arange(
        time_points[0],
        len(time_points) + steady_state_time,
        time_points[1] - time_points[0]
    )
    size = 200
    num_iterations = 10

2.5799999999999987


Running Variance Ratio Simulations:   0%|          | 0/42 [00:00<?, ?it/s]

Attempt 1/10


/home/ianyang/stochastic_simulations/src/simulation/mean_var_autocorr.py:32: RuntimeWarning: invalid value encountered in multiply
  ACmRNA_eq = sp.exp(-d * t) * (
/home/ianyang/stochastic_simulations/src/simulation/mean_var_autocorr.py:33: RuntimeWarning: invalid value encountered in multiply
  d * sp.exp((d - sigma_u - sigma_b) * t) * rho * sigma_u
/home/ianyang/stochastic_simulations/src/simulation/mean_var_autocorr.py:164: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  solution = fsolve(
/home/ianyang/stochastic_simulations/src/simulation/mean_var_autocorr.py:164: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last five Jacobian evaluations.
  solution = fsolve(
/home/ianyang/stochastic_simulations/src/simulation/mean_var_autocorr.py:164: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  solution = fsolve(


Attempt 2/10
[STRESS] ✅ Found: {'rho': 5795.902957821557, 'sigma_b': 0.022843952095094676, 'd': 0.7346306158352103}
Attempt 1/10


Running Variance Ratio Simulations:   2%|▏         | 1/42 [04:00<2:44:20, 240.50s/it]

[NORMAL] ✅ Found: {'rho': 1179.1338138131455, 'sigma_b': 0.06014543999856846, 'd': 0.7827636158617561}
Updated Parameter Sets: [{'sigma_u': 18.0, 'sigma_b': 0.022843952095094676, 'rho': 5795.902957821557, 'd': 0.7346306158352103, 'label': 0}, {'sigma_u': 9.0, 'sigma_b': 0.06014543999856846, 'rho': 1179.1338138131455, 'd': 0.7827636158617561, 'label': 1}]
Attempt 1/10


Running Variance Ratio Simulations:   2%|▏         | 1/42 [04:25<3:01:22, 265.44s/it]


KeyboardInterrupt: 

In [ ]:
###### var_v_accuracy_12_04_2025.py ######
#!/usr/bin/env python3
import numpy as np
import pandas as pd
import os
import tqdm
# from sympy import sqrt
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# from sklearn.decomposition import PCA
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# import sympy as sp
# from sympy import init_printing, solve
# from scipy.optimize import fsolve

# Import your own local modules/functions
from simulation.simulate_telegraph_model import simulate_two_telegraph_model_systems
from stats.report import statistical_report

from utils.load_data import load_and_split_data
from utils.steady_state import save_steady_state
from simulation.mean_var_autocorr import find_parameters
from classifiers.lstm_classifier import lstm_classifier
from classifiers.lstm_conv1d_classifier import lstm_conv1d_classifier
from classifiers.svm_classifier import svm_classifier
from classifiers.random_forest_classifier import random_forest_classifier
from classifiers.logistic_regression_classifier import logistic_regression_classifier
from classifiers.mlp_classifier import mlp_classifier
from classifiers.random_classifier import random_classifier

############# READ THIS ################
# 1. After finetuning the LSTM architercture and hyperparameters, I am now running some classification benchmarks using the new model (saved in the folder 'src/classifiers/lstm_conv1d_classifier.py').: 
# hidden_size      64.000
# num_layers        2.000
# dropout_rate      0.010
# learning_rate     0.001
# batch_size       32.000
# 2. We want ONLY the steady state part of the data for the classification task, we use save_steady_state() to get the steady state part of the data.
# 3. We want to extend the time points to get enough steady state data.

############# READ THIS ################


###############################################################################
# 1) Define target mean, variance and autocorrelations, and some parameters to start with
###############################################################################
variance_target_normal = 1200.0  # Fixed normal variance
mu_target = 10.0                 # Mean (same for both)
variance_ratios = np.arange(0.1, 3.0, 0.01)
autocorr_target = 0.5

parameters = {
    "stress": {"sigma_u": 18.0},
    "normal": {"sigma_u": 9.0}
}

###############################################################################
# 2) Loop over different variance ratios
###############################################################################
for ratio in tqdm.tqdm(variance_ratios, desc="Running Variance Ratio Simulations"):
    # For the stress condition, we define variance_target_stress by ratio
    variance_target_stress = ratio * variance_target_normal

    # store results for each condition
    results = {}
    for condition, param in parameters.items():
        # Decide which variance to use for this condition
        if condition == "normal":
            var_for_condition = variance_target_normal
        else:  # condition == "stress"
            var_for_condition = variance_target_stress

        try:
            # Fix all three statistical properties
            rho, sigma_b, d = find_parameters(
                param, mu_target, var_for_condition, autocorr_target
            )

            results[condition] = {"rho": rho, "sigma_b": sigma_b, "d": d}
            print(f"[{condition.upper()}] ✅ Found: {results[condition]}")
        except ValueError as e:
            print(f'{e}')
            print(f"[{condition.upper()}] ❌ No suitable solution found.")


    # update parameter sets 
    parameter_sets = [
        {"sigma_u": parameters["stress"]["sigma_u"], 
        "sigma_b": results["stress"]['sigma_b'], 
        "rho": results["stress"]['rho'], 
        "d": results["stress"]['d'], 
        "label": 0},
        
        {"sigma_u": parameters["normal"]["sigma_u"], 
        "sigma_b": results["normal"]['sigma_b'], 
        "rho": results["normal"]['rho'], 
        "d": results["normal"]['d'], 
        "label": 1}
    ]

    # Output the results
    print("Updated Parameter Sets:", parameter_sets)

    # Simulation parameters
    min_d = min(pset['d'] for pset in parameter_sets)
    steady_state_time = int(10 / min_d)
    time_points = np.arange(0, 144.0, 1.0)
    extended_time_points = np.arange(
        time_points[0],
        len(time_points) + steady_state_time,
        time_points[1] - time_points[0]
    )
    size = 200
    num_iterations = 10

    ###########################################################################
    # 3) Simulate & Save data
    ###########################################################################
    for i in range(num_iterations):
        df_results = simulate_two_telegraph_model_systems(parameter_sets, extended_time_points, size)

        output_dir = f"data_12_04_2025/mRNA_trajectories_variance_{int(variance_target_stress)}_{int(variance_target_normal)}"
        os.makedirs(output_dir, exist_ok=True)
        # save full time series
        output_file = f"{output_dir}/m_traj_{variance_target_stress}_{variance_target_normal}_{i}.csv"
        df_results.to_csv(output_file, index=False)

        # get only the steady state part of the data
        save_path = f'{output_dir}/steady_state_trajectories/'
        remaining_time_points, steady_state_series = save_steady_state(output_file, parameter_sets, time_points,
                                                                       save_path=save_path,)

        #######################################################################
        # 4) Analysis & classification (optional, can be done post-hoc)
        #######################################################################
        stress_trajectories = steady_state_series[steady_state_series['label'] == 0].iloc[:, 1:].values
        normal_trajectories = steady_state_series[steady_state_series['label'] == 1].iloc[:, 1:].values
        stats = statistical_report(parameter_sets, stress_trajectories, normal_trajectories)

        # read in the steady state data
        steady_state_file = os.path.join(save_path, f"{os.path.splitext(os.path.basename(output_file))[0]}_SS.csv") # this is the file name we saved it as

        # classifiers
        X_train, X_val, X_test, y_train, y_val, y_test = load_and_split_data(steady_state_file, split_val_size=0.2) # we must define split_val_size here to get a validation set
        svm_rbf_accuracy = svm_classifier(X_train, X_test, y_train, y_test)
        svm_linear_accuracy = svm_classifier(X_train, X_test, y_train, y_test, svm_kernel='linear')
        rf_accuracy = random_forest_classifier(X_train, X_test, y_train, y_test)
        log_reg_accuracy = logistic_regression_classifier(X_train, X_test, y_train, y_test)
        mlp_accuracy = mlp_classifier(X_train, X_val, X_test, y_train, y_val, y_test, epochs=100)
        random_accuracy = random_classifier(y_test)
        lstm_accuracy = lstm_classifier(X_train, X_val, X_test, y_train, y_val, y_test, epochs=50)
        lstm_conv1d_accuracy = lstm_conv1d_classifier(X_train, X_val, X_test, y_train, y_val, y_test, epochs=50)

        df_acc_results = pd.DataFrame({
            "Parameter Sets": [parameter_sets],
            "Stats": [stats],
            "Variance Ratio": [ratio],
            "SVM (rbf) Accuracy": [svm_rbf_accuracy],
            "SVM (linear) Accuracy": [svm_linear_accuracy],
            "Random Forest Accuracy": [rf_accuracy],
            "Logistic Regression Accuracy": [log_reg_accuracy],
            "MLP Accuracy": [mlp_accuracy],
            "Random Classifier Accuracy": [random_accuracy],
            "LSTM Accuracy": [lstm_accuracy],
            "LSTM Conv1D Accuracy": [lstm_conv1d_accuracy]
        })

        # Save the accuracy results to a CSV file
        if not os.path.isfile("data_12_04_2025/accuracy_results_12_04_2025.csv"):
            df_acc_results.to_csv("data_12_04_2025/accuracy_results_12_04_2025.csv", index=False)
        else:
            df_acc_results.to_csv("data_12_04_2025/accuracy_results_12_04_2025.csv", mode='a', header=False, index=False)

In [2]:
from simulation.mean_var_autocorr import check_biological_appropriateness, find_parameters
import numpy as np
import tqdm
%load_ext autoreload
%autoreload 2

# Define a function to try different values
def find_biologically_appropriate_parameters(target_cv_initial=1.7, mu_target_initial=5, max_attempts=10):
    # Try different combinations of CV and mean until we find biologically appropriate values
    for attempt in range(max_attempts):
        # On first attempt, use the initial values
        if attempt == 0:
            target_cv = target_cv_initial
            mu = mu_target_initial
        # On subsequent attempts, adjust the values
        else:
            # Try reducing CV or increasing mean to improve Fano factor and CV
            target_cv = target_cv_initial * (0.9 - attempt * 0.05)  # Gradually reduce CV
            mu = mu_target_initial * (1.1 + attempt * 0.1)         # Gradually increase mean
        
        # Calculate corresponding variance
        variance = (target_cv * mu)**2
        
        print(f"\nAttempt {attempt+1}/{max_attempts}")
        print(f"Testing: CV = {target_cv:.2f}, Mean = {mu:.2f}, Variance = {variance:.2f}")
        
        # Check if these values are biologically appropriate
        is_appropriate = check_biological_appropriateness(variance, mu)
        
        if is_appropriate:
            print(f"✅ Found biologically appropriate values: CV = {target_cv:.2f}, Mean = {mu:.2f}, Variance = {variance:.2f}")
            return target_cv, mu, variance
    
    # If we reach here, we couldn't find appropriate parameters
    print("❌ Could not find biologically appropriate parameters")
    return None, None, None

# Function to pre-check all CV ratios for biological appropriateness
def find_parameters_without_skipping(initial_cv_normal=1.7, initial_mu_target=5, cv_ratios=np.arange(0.1, 3.0, 0.01), max_attempts=5):
    for attempt in range(max_attempts):
        # On first attempt, use the initial values
        if attempt == 0:
            cv_normal = initial_cv_normal
            mu = initial_mu_target
        # On subsequent attempts, adjust the values
        else:
            # Try decreasing CV and increasing mean to improve all conditions
            cv_normal = initial_cv_normal * (0.85 - attempt * 0.05)  # More aggressive CV reduction
            mu = initial_mu_target * (1.3 + attempt * 0.2)          # More aggressive mean increase
        
        # Calculate normal variance
        variance_normal = (cv_normal * mu)**2
        
        print(f"\nTesting parameters: CV normal = {cv_normal:.2f}, Mean = {mu:.2f}")
        
        # Check if normal condition is biologically appropriate
        is_normal_appropriate = check_biological_appropriateness(variance_normal, mu)
        if not is_normal_appropriate:
            print(f"⚠️ Normal condition not biologically appropriate. Trying different parameters...")
            continue
        
        # Check all CV ratios for the stress condition
        all_appropriate = True
        inappropriate_ratios = []
        
        for ratio in tqdm.tqdm(cv_ratios, desc="Checking all CV ratios"):
            cv_stress = ratio * cv_normal
            variance_stress = (cv_stress * mu)**2
            
            is_stress_appropriate = check_biological_appropriateness(variance_stress, mu)
            if not is_stress_appropriate:
                all_appropriate = False
                inappropriate_ratios.append(ratio)
        
        if all_appropriate:
            print(f"✅ Found parameters where all CV ratios are biologically appropriate!")
            print(f"CV normal = {cv_normal:.2f}, Mean = {mu:.2f}, Variance normal = {variance_normal:.2f}")
            return cv_normal, mu, variance_normal
        else:
            print(f"⚠️ {len(inappropriate_ratios)} out of {len(cv_ratios)} CV ratios are not biologically appropriate")
            print(f"First few inappropriate ratios: {inappropriate_ratios[:5]}")
    
    print("❌ Could not find parameters where all CV ratios are biologically appropriate")
    print("You may need to adjust the range of CV ratios or try more extreme parameter values")
    return None, None, None

# Range of CV ratios to test (stress CV / normal CV)
cv_ratio = np.arange(0.1, 3.0, 0.01)  

# Set initial parameters for search
initial_cv_normal = 0.8  # Try a lower initial CV for normal condition
initial_mu_target = 15   # Try a higher initial mean
autocorr_target = 0.5    # Autocorrelation (same for both conditions)

# Find biologically appropriate parameters for normal condition
target_cv_normal, mu_target, variance_normal = find_biologically_appropriate_parameters(
    target_cv_initial=initial_cv_normal, 
    mu_target_initial=initial_mu_target
)

# Range of CV ratios to test (stress CV / normal CV)
cv_ratio = np.arange(0.1, 3.0, 0.01)  

# Initial parameters
parameters = {
    "stress": {"sigma_u": 18.0},
    "normal": {"sigma_u": 9.0}
}

print(f"\nStarting simulations with:")
print(f"Normal CV: {target_cv_normal:.2f}, Mean: {mu_target:.2f}, Variance: {variance_normal:.2f}")

###############################################################################
# 2) Loop over different CV ratios
###############################################################################

for ratio in tqdm.tqdm(cv_ratio, desc="Running CV Ratio Simulations"):
    # For the stress condition, we define CV by ratio
    target_cv_stress = ratio * target_cv_normal
    
    # Calculate corresponding variance values for reporting only
    variance_stress = (target_cv_stress * mu_target)**2
    
    print(f"\nTesting CV ratio: {ratio:.2f}, Stress CV: {target_cv_stress:.2f}, Normal CV: {target_cv_normal:.2f}")
    print(f"Corresponding variances - Stress: {variance_stress:.2f}, Normal: {variance_normal:.2f}")
    
    # Check if stress parameters are biologically appropriate
    is_stress_appropriate = check_biological_appropriateness(variance_stress, mu_target)
    
    # If stress parameters are not biologically appropriate, skip this iteration
    if not is_stress_appropriate:
        print(f"⚠️ Stress parameters are not biologically appropriate for ratio {ratio:.2f}. Skipping.")
        continue

    # Store results for each condition
    results = {}
    success = True  # Track if both conditions succeed
    
    for condition, param in parameters.items():
        # Decide which CV to use for this condition
        if condition == "normal":
            cv_for_condition = target_cv_normal
        else:  # condition == "stress"
            cv_for_condition = target_cv_stress

        try:
            # Fix mean, CV, and autocorrelation
            rho, sigma_b, d = find_parameters(
                param, mu_target=mu_target, variance_target=None, autocorr_target=autocorr_target, 
                cv_target=cv_for_condition, check_biological=True
            )

            results[condition] = {"rho": rho, "sigma_b": sigma_b, "d": d}
            print(f"[{condition.upper()}] ✅ Found: {results[condition]}")
        except ValueError as e:
            print(f'{e}')
            print(f"[{condition.upper()}] ❌ No suitable solution found.")
            success = False
    
    # If any condition failed, skip to the next ratio
    if not success:
        print(f"Skipping ratio {ratio:.2f} due to parameter finding failure")
        continue

    # update parameter sets 
    parameter_sets = [
        {"sigma_u": parameters["stress"]["sigma_u"], 
        "sigma_b": results["stress"]['sigma_b'], 
        "rho": results["stress"]['rho'], 
        "d": results["stress"]['d'], 
        "label": 0},
        
        {"sigma_u": parameters["normal"]["sigma_u"], 
        "sigma_b": results["normal"]['sigma_b'], 
        "rho": results["normal"]['rho'], 
        "d": results["normal"]['d'], 
        "label": 1}
    ]

    # Output the results
    print("Updated Parameter Sets:", parameter_sets)

# Pre-check parameters for all CV ratios
find_parameters_without_skipping(
    initial_cv_normal=target_cv_normal, 
    initial_mu_target=mu_target, 
    cv_ratios=cv_ratio, 
    max_attempts=5
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Attempt 1/10
Testing: CV = 0.80, Mean = 15.00, Variance = 144.00
✅ System is biologically appropriate with Fano factor: 9.60, CV: 0.80
✅ Found biologically appropriate values: CV = 0.80, Mean = 15.00, Variance = 144.00

Starting simulations with:
Normal CV: 0.80, Mean: 15.00, Variance: 144.00


Running CV Ratio Simulations:   0%|          | 0/290 [00:00<?, ?it/s]


Testing CV ratio: 0.10, Stress CV: 0.08, Normal CV: 0.80
Corresponding variances - Stress: 1.44, Normal: 144.00
⚠️ WARNING: Fano factor 0.10 < 1, consider changing the target parameters.
⚠️ Stress parameters are not biologically appropriate for ratio 0.10. Skipping.

Testing CV ratio: 0.11, Stress CV: 0.09, Normal CV: 0.80
Corresponding variances - Stress: 1.74, Normal: 144.00
⚠️ WARNING: Fano factor 0.12 < 1, consider changing the target parameters.
⚠️ Stress parameters are not biologically appropriate for ratio 0.11. Skipping.

Testing CV ratio: 0.12, Stress CV: 0.10, Normal CV: 0.80
Corresponding variances - Stress: 2.07, Normal: 144.00
⚠️ WARNING: Fano factor 0.14 < 1, consider changing the target parameters.
⚠️ Stress parameters are not biologically appropriate for ratio 0.12. Skipping.

Testing CV ratio: 0.13, Stress CV: 0.10, Normal CV: 0.80
Corresponding variances - Stress: 2.43, Normal: 144.00
⚠️ WARNING: Fano factor 0.16 < 1, consider changing the target parameters.
⚠️ Stres

Running CV Ratio Simulations:   8%|▊         | 23/290 [00:05<01:08,  3.88it/s]



KeyboardInterrupt: 

In [4]:
cv_ratio = np.arange(0.5, 2.0, 0.01)  

# Try to find parameters that work for all CV ratios
print("Finding parameters where all CV ratios are biologically appropriate...")
target_cv_normal, mu_target, variance_normal = find_parameters_without_skipping(
    initial_cv_normal=initial_cv_normal, 
    initial_mu_target=initial_mu_target,
    cv_ratios=cv_ratio,
    max_attempts=10  # Increase max attempts to try more parameter combinations
)

# If we couldn't find parameters that work for all ratios, use the original function
if target_cv_normal is None:
    print("\nFalling back to finding biologically appropriate parameters for normal condition only...")
    target_cv_normal, mu_target, variance_normal = find_biologically_appropriate_parameters(
        target_cv_initial=initial_cv_normal, 
        mu_target_initial=initial_mu_target
    )

# Initial parameters for telegraph model
parameters = {
    "stress": {"sigma_u": 18.0},
    "normal": {"sigma_u": 9.0}
}

print(f"\nStarting simulations with:")
print(f"Normal CV: {target_cv_normal:.2f}, Mean: {mu_target:.2f}, Variance: {variance_normal:.2f}")

# If only some ratios were inappropriate, you could narrow down the range
# For example:
# cv_ratio = np.arange(0.5, 2.0, 0.01)  # Example of a narrower range if needed

Finding parameters where all CV ratios are biologically appropriate...

Testing parameters: CV normal = 0.80, Mean = 15.00
✅ System is biologically appropriate with Fano factor: 9.60, CV: 0.80


Checking all CV ratios: 100%|██████████| 150/150 [00:00<00:00, 132591.28it/s]


✅ System is biologically appropriate with Fano factor: 2.40, CV: 0.40
✅ System is biologically appropriate with Fano factor: 2.50, CV: 0.41
✅ System is biologically appropriate with Fano factor: 2.60, CV: 0.42
✅ System is biologically appropriate with Fano factor: 2.70, CV: 0.42
✅ System is biologically appropriate with Fano factor: 2.80, CV: 0.43
✅ System is biologically appropriate with Fano factor: 2.90, CV: 0.44
✅ System is biologically appropriate with Fano factor: 3.01, CV: 0.45
✅ System is biologically appropriate with Fano factor: 3.12, CV: 0.46
✅ System is biologically appropriate with Fano factor: 3.23, CV: 0.46
✅ System is biologically appropriate with Fano factor: 3.34, CV: 0.47
✅ System is biologically appropriate with Fano factor: 3.46, CV: 0.48
✅ System is biologically appropriate with Fano factor: 3.57, CV: 0.49
✅ System is biologically appropriate with Fano factor: 3.69, CV: 0.50
✅ System is biologically appropriate with Fano factor: 3.81, CV: 0.50
✅ System is biologic

Checking all CV ratios: 100%|██████████| 150/150 [00:00<00:00, 147582.83it/s]


✅ System is biologically appropriate with Fano factor: 2.30, CV: 0.32
✅ System is biologically appropriate with Fano factor: 2.40, CV: 0.33
✅ System is biologically appropriate with Fano factor: 2.49, CV: 0.33
✅ System is biologically appropriate with Fano factor: 2.59, CV: 0.34
✅ System is biologically appropriate with Fano factor: 2.69, CV: 0.35
✅ System is biologically appropriate with Fano factor: 2.79, CV: 0.35
✅ System is biologically appropriate with Fano factor: 2.89, CV: 0.36
✅ System is biologically appropriate with Fano factor: 2.99, CV: 0.36
✅ System is biologically appropriate with Fano factor: 3.10, CV: 0.37
✅ System is biologically appropriate with Fano factor: 3.21, CV: 0.38
✅ System is biologically appropriate with Fano factor: 3.32, CV: 0.38
✅ System is biologically appropriate with Fano factor: 3.43, CV: 0.39
✅ System is biologically appropriate with Fano factor: 3.54, CV: 0.40
✅ System is biologically appropriate with Fano factor: 3.66, CV: 0.40
✅ System is biologic

...

✅ System is biologically appropriate with Fano factor: 1.62, CV: 0.20
✅ System is biologically appropriate with Fano factor: 1.69, CV: 0.20
✅ System is biologically appropriate with Fano factor: 1.75, CV: 0.21
✅ System is biologically appropriate with Fano factor: 1.82, CV: 0.21
✅ System is biologically appropriate with Fano factor: 1.89, CV: 0.22
✅ System is biologically appropriate with Fano factor: 1.96, CV: 0.22
✅ System is biologically appropriate with Fano factor: 2.03, CV: 0.22
✅ System is biologically appropriate with Fano factor: 2.11, CV: 0.23
✅ System is biologically appropriate with Fano factor: 2.18, CV: 0.23
✅ System is biologically appropriate with Fano factor: 2.26, CV: 0.24
✅ System is biologically appropriate with Fano factor: 2.33, CV: 0.24
✅ System is biologically appropriate with Fano factor: 2.41, CV: 0.24
✅ System is biologically appropriate with Fano factor: 2.49, CV: 0.25
✅ System is biologically appropriate with Fano factor: 2.57, CV: 0.25
✅ System is biologic

Checking all CV ratios: 100%|██████████| 150/150 [00:00<00:00, 91712.19it/s]


✅ System is biologically appropriate with Fano factor: 1.41, CV: 0.18
✅ System is biologically appropriate with Fano factor: 1.47, CV: 0.18
✅ System is biologically appropriate with Fano factor: 1.52, CV: 0.19
✅ System is biologically appropriate with Fano factor: 1.58, CV: 0.19
✅ System is biologically appropriate with Fano factor: 1.64, CV: 0.19
✅ System is biologically appropriate with Fano factor: 1.71, CV: 0.20
✅ System is biologically appropriate with Fano factor: 1.77, CV: 0.20
✅ System is biologically appropriate with Fano factor: 1.83, CV: 0.21
✅ System is biologically appropriate with Fano factor: 1.90, CV: 0.21
✅ System is biologically appropriate with Fano factor: 1.96, CV: 0.21
✅ System is biologically appropriate with Fano factor: 2.03, CV: 0.22
✅ System is biologically appropriate with Fano factor: 2.10, CV: 0.22
✅ System is biologically appropriate with Fano factor: 2.17, CV: 0.22
✅ System is biologically appropriate with Fano factor: 2.24, CV: 0.23
✅ System is biologic

Checking all CV ratios: 100%|██████████| 150/150 [00:00<00:00, 147409.93it/s]

✅ System is biologically appropriate with Fano factor: 1.19, CV: 0.16
✅ System is biologically appropriate with Fano factor: 1.24, CV: 0.16
✅ System is biologically appropriate with Fano factor: 1.29, CV: 0.17
✅ System is biologically appropriate with Fano factor: 1.34, CV: 0.17
✅ System is biologically appropriate with Fano factor: 1.39, CV: 0.17
✅ System is biologically appropriate with Fano factor: 1.44, CV: 0.18
✅ System is biologically appropriate with Fano factor: 1.49, CV: 0.18
✅ System is biologically appropriate with Fano factor: 1.55, CV: 0.18
✅ System is biologically appropriate with Fano factor: 1.60, CV: 0.19
✅ System is biologically appropriate with Fano factor: 1.66, CV: 0.19
✅ System is biologically appropriate with Fano factor: 1.71, CV: 0.19
✅ System is biologically appropriate with Fano factor: 1.77, CV: 0.20
✅ System is biologically appropriate with Fano factor: 1.83, CV: 0.20
✅ System is biologically appropriate with Fano factor: 1.89, CV: 0.20
✅ System is biologic

In [5]:
###############################################################################
# 2) Loop over different CV ratios with optimized parameters
###############################################################################

# Now we can run the CV ratio loop with confidence that we've found parameters
# that should work for all ratios
inappropriate_ratios = []

for ratio in tqdm.tqdm(cv_ratio, desc="Running CV Ratio Simulations"):
    # For the stress condition, we define CV by ratio
    target_cv_stress = ratio * target_cv_normal
    
    # Calculate corresponding variance values for reporting only
    variance_stress = (target_cv_stress * mu_target)**2
    
    print(f"\nTesting CV ratio: {ratio:.2f}, Stress CV: {target_cv_stress:.2f}, Normal CV: {target_cv_normal:.2f}")
    print(f"Corresponding variances - Stress: {variance_stress:.2f}, Normal: {variance_normal:.2f}")
    
    # Double-check if stress parameters are biologically appropriate
    is_stress_appropriate = check_biological_appropriateness(variance_stress, mu_target)
    
    # If stress parameters are not biologically appropriate, track it and skip
    if not is_stress_appropriate:
        print(f"⚠️ Stress parameters are not biologically appropriate for ratio {ratio:.2f}. Skipping.")
        inappropriate_ratios.append(ratio)
        continue

    # Store results for each condition
    results = {}
    success = True  # Track if both conditions succeed
    
    for condition, param in parameters.items():
        # Decide which CV to use for this condition
        if condition == "normal":
            cv_for_condition = target_cv_normal
        else:  # condition == "stress"
            cv_for_condition = target_cv_stress

        try:
            # Fix mean, CV, and autocorrelation
            rho, sigma_b, d = find_parameters(
                param, mu_target=mu_target, variance_target=None, autocorr_target=autocorr_target, 
                cv_target=cv_for_condition, check_biological=True
            )

            results[condition] = {"rho": rho, "sigma_b": sigma_b, "d": d}
            print(f"[{condition.upper()}] ✅ Found: {results[condition]}")
        except ValueError as e:
            print(f'{e}')
            print(f"[{condition.upper()}] ❌ No suitable solution found.")
            success = False
    
    # If any condition failed, skip to the next ratio
    if not success:
        print(f"Skipping ratio {ratio:.2f} due to parameter finding failure")
        continue

    # update parameter sets 
    parameter_sets = [
        {"sigma_u": parameters["stress"]["sigma_u"], 
        "sigma_b": results["stress"]['sigma_b'], 
        "rho": results["stress"]['rho'], 
        "d": results["stress"]['d'], 
        "label": 0},
        
        {"sigma_u": parameters["normal"]["sigma_u"], 
        "sigma_b": results["normal"]['sigma_b'], 
        "rho": results["normal"]['rho'], 
        "d": results["normal"]['d'], 
        "label": 1}
    ]

    # Output the results
    print("Updated Parameter Sets:", parameter_sets)

# Summarize results
if len(inappropriate_ratios) == 0:
    print("\n✅ All CV ratios were biologically appropriate! No iterations were skipped.")
else:
    print(f"\n⚠️ {len(inappropriate_ratios)} out of {len(cv_ratio)} CV ratios were not biologically appropriate")
    print(f"Inappropriate ratios: {inappropriate_ratios[:10]}{'...' if len(inappropriate_ratios) > 10 else ''}")
    
    # Suggest parameter adjustment if needed
    if len(inappropriate_ratios) > len(cv_ratio) * 0.2:  # If more than 20% are inappropriate
        print("\nConsider trying again with different initial parameters:")
        print("1. Lower CV values (e.g., initial_cv_normal = 0.5)")
        print("2. Higher mean values (e.g., initial_mu_target = 20)")
        print("3. Narrowing your CV ratio range (e.g., cv_ratio = np.arange(0.5, 1.5, 0.01))")
    else:
        print("\nThe majority of CV ratios are biologically appropriate. You may proceed with these parameters.")


Running CV Ratio Simulations:   0%|          | 0/150 [00:00<?, ?it/s]


Testing CV ratio: 0.50, Stress CV: 0.16, Normal CV: 0.32
Corresponding variances - Stress: 55.35, Normal: 221.41
✅ System is biologically appropriate with Fano factor: 1.19, CV: 0.16
Attempt 1/10
✅ System is biologically appropriate with Fano factor: 1.19, CV: 0.16
[STRESS] ✅ Found: {'rho': 44.764498238863645, 'sigma_b': 46.7068330777153, 'd': 0.6948819719012493}
Attempt 1/10
✅ System is biologically appropriate with Fano factor: 1.19, CV: 0.16
[STRESS] ✅ Found: {'rho': 44.764498238863645, 'sigma_b': 46.7068330777153, 'd': 0.6948819719012493}
Attempt 1/10


Running CV Ratio Simulations:   1%|          | 1/150 [00:02<05:19,  2.14s/it]

✅ System is biologically appropriate with Fano factor: 4.76, CV: 0.32
[NORMAL] ✅ Found: {'rho': 91.0994661910079, 'sigma_b': 5.4008188964845685, 'd': 0.734742742570821}
Updated Parameter Sets: [{'sigma_u': 18.0, 'sigma_b': 46.7068330777153, 'rho': 44.764498238863645, 'd': 0.6948819719012493, 'label': 0}, {'sigma_u': 9.0, 'sigma_b': 5.4008188964845685, 'rho': 91.0994661910079, 'd': 0.734742742570821, 'label': 1}]

Testing CV ratio: 0.51, Stress CV: 0.16, Normal CV: 0.32
Corresponding variances - Stress: 57.59, Normal: 221.41
✅ System is biologically appropriate with Fano factor: 1.24, CV: 0.16
Attempt 1/10
✅ System is biologically appropriate with Fano factor: 1.24, CV: 0.16
[STRESS] ✅ Found: {'rho': 46.55888774648159, 'sigma_b': 40.93224711364454, 'd': 0.6954441048779944}
Attempt 1/10
✅ System is biologically appropriate with Fano factor: 1.24, CV: 0.16
[STRESS] ✅ Found: {'rho': 46.55888774648159, 'sigma_b': 40.93224711364454, 'd': 0.6954441048779944}
Attempt 1/10


Running CV Ratio Simulations:   1%|▏         | 2/150 [00:11<15:26,  6.26s/it]

...

✅ System is biologically appropriate with Fano factor: 1.34, CV: 0.17
[STRESS] ✅ Found: {'rho': 49.92733723384751, 'sigma_b': 33.25255155831636, 'd': 0.6966183913491262}
Attempt 1/10


Running CV Ratio Simulations:   3%|▎         | 4/150 [00:25<17:37,  7.24s/it]

✅ System is biologically appropriate with Fano factor: 4.76, CV: 0.32
[NORMAL] ✅ Found: {'rho': 91.09946619100955, 'sigma_b': 5.4008188964834165, 'd': 0.7347427425708325}
Updated Parameter Sets: [{'sigma_u': 18.0, 'sigma_b': 33.25255155831636, 'rho': 49.92733723384751, 'd': 0.6966183913491262, 'label': 0}, {'sigma_u': 9.0, 'sigma_b': 5.4008188964834165, 'rho': 91.09946619100955, 'd': 0.7347427425708325, 'label': 1}]

Testing CV ratio: 0.54, Stress CV: 0.17, Normal CV: 0.32
Corresponding variances - Stress: 64.56, Normal: 221.41
✅ System is biologically appropriate with Fano factor: 1.39, CV: 0.17
Attempt 1/10
✅ System is biologically appropriate with Fano factor: 1.39, CV: 0.17
[STRESS] ✅ Found: {'rho': 51.54101545193121, 'sigma_b': 30.520905547325437, 'd': 0.6972179108068793}
Attempt 1/10
✅ System is biologically appropriate with Fano factor: 1.39, CV: 0.17
[STRESS] ✅ Found: {'rho': 51.54101545193121, 'sigma_b': 30.520905547325437, 'd': 0.6972179108068793}
Attempt 1/10


Running CV Ratio Simulations:   3%|▎         | 4/150 [00:27<16:47,  6.90s/it]



KeyboardInterrupt: 

In [3]:
# Test the modified find_parameters function with Jacobian validation
from simulation.mean_var_autocorr import find_parameters, validate_jacobian
import numpy as np

print("Testing Jacobian validation...")

# Define test parameters
test_params = {"sigma_u": 9.0}
mu_target = 10.0
variance_target = 120.0
autocorr_target = 0.5

# First, just validate the Jacobian separately
# diff = validate_jacobian(
    # sigma_u_val=test_params["sigma_u"],
    # mu_target=mu_target,
    # variance_target=variance_target,
    # autocorr_target=autocorr_target
# )

# print(f"Jacobian validation difference: {diff:.6e}")

# Now try using find_parameters with a simple test case
print("\nTesting find_parameters with Jacobian validation...")
try:
    rho, sigma_b, d = find_parameters(
        test_params,
        mu_target=mu_target,
        variance_target=variance_target,
        autocorr_target=autocorr_target,
        num_guesses=500  # Reduce for quicker testing
    )
    print(f"✅ Parameters found successfully:")
    print(f"  rho = {rho:.6f}")
    print(f"  sigma_b = {sigma_b:.6f}")
    print(f"  d = {d:.6f}")
    
    # Check if the found parameters actually satisfy our equations
    from simulation.mean_var_autocorr import equations
    residuals = equations([rho, sigma_b, d], test_params["sigma_u"], mu_target, variance_target, autocorr_target)
    print(f"\nResiduals: {[f'{r:.6e}' for r in residuals]}")
    
except ValueError as e:
    print(f"❌ Error: {e}")

# Test with a problematic case (use the ratio that was causing issues)
print("\nTesting with previously problematic ratio...")
problem_ratio = 2.57  # This was identified as problematic earlier

target_cv = 0.8  # From previous results
mu = 15.0        # From previous results
problem_cv = problem_ratio * target_cv
problem_variance = (problem_cv * mu)**2

print(f"Problem case - CV: {problem_cv:.2f}, Mean: {mu:.2f}, Variance: {problem_variance:.2f}")

try:
    rho, sigma_b, d = find_parameters(
        test_params,
        mu_target=mu,
        variance_target=None,  
        autocorr_target=autocorr_target,
        cv_target=problem_cv,
        num_guesses=1000
    )
    print(f"✅ Parameters found successfully for problematic case:")
    print(f"  rho = {rho:.6f}")
    print(f"  sigma_b = {sigma_b:.6f}")
    print(f"  d = {d:.6f}")
    
except ValueError as e:
    print(f"❌ Error with problematic case: {e}")

Testing Jacobian validation...

Testing find_parameters with Jacobian validation...
⚠️ Jacobian validation warning: difference = 2.427476e-04 > 0.0001
Attempt 1/10
✅ System is biologically appropriate with Fano factor: 12.00, CV: 1.10
✅ Parameters found successfully:
  rho = 121.847482
  sigma_b = 0.607089
  d = 0.769976

Residuals: ['4.440892e-13', '5.940137e-12', '7.771561e-16']

Testing with previously problematic ratio...
Problem case - CV: 2.06, Mean: 15.00, Variance: 951.11
✅ Jacobian validation passed: difference = 3.285721e-06
Attempt 1/10
⚠️ WARNING: Fano factor 63.41 > 20, consider changing the target parameters.
⚠️ WARNING: Fano factor 63.41 > 20, consider changing the target parameters.
⚠️ WARNING: Fano factor 63.41 > 20, consider changing the target parameters.
⚠️ WARNING: Fano factor 63.41 > 20, consider changing the target parameters.
⚠️ WARNING: Fano factor 63.41 > 20, consider changing the target parameters.
⚠️ WARNING: Fano factor 63.41 > 20, consider changing the tar

KeyboardInterrupt: 